In [ ]:
# imports

import os
import pandas as pd
import chardet
import random
import csv
from gpx_converter import Converter
import re

In [ ]:
# variables
startdir = r'D:\Bachelorarbeit\Dataset\created_Dataset\xlsx_files'
file_names = []
outputdir = r'D:\Bachelorarbeit\Dataset\created_Dataset\xlsx_files'

In [ ]:
# get all file paths

for path, subdir, files in os.walk(startdir):
    for file in files:
        file_names.append(startdir + "\\" + file)
file_names

In [ ]:
# get encoding of files
def get_encoding(filepath):
    with open(filepath, "rb") as f:
        content = f.read()
    return chardet.detect(content)['encoding']
encoding = get_encoding(file_names[0])

# cp1252 for ISO-8859-1

In [ ]:
# get separator
def get_separator(filepath):
    with open(filepath, 'r') as csvfile:
        delimiter = str(csv.Sniffer().sniff(csvfile.read()).delimiter)
        return delimiter
separator = get_separator(file_names[0])

In [ ]:
# split files with pandas

for file in file_names:
    i = 0
    sep = get_separator(file)
    df = pd.read_csv(file,sep=sep, encoding='ascii')
    
    while len(df) > 700:
        i += 1
        # split into 500 - 700 lines randomly
        num = random.randint(500, 700)
        df_new = df.iloc[:num]
        df = df.iloc[num:]
        
        # output files
        outp = outputdir +'\\'+ file.split("\\")[-1].split(".")[0] + str(i) + '.csv'
        df_new.to_csv(outp)
        
        if len(df) <= 500:
            outp = outputdir +'\\' + file.split("\\")[-1].split(".")[0] + '0' + '.csv'
            df.to_csv(outp)
            break
        
    

In [ ]:
# split data into folders

inp = r'C:\Users\achte\Documents\KI_Ba\DS_test\gps\IOVNDB_new'
outp1 = r'C:\Users\achte\Documents\KI_Ba\DS_test\gps\csv'
outp2 = r'C:\Users\achte\Documents\KI_Ba\DS_test\gps\xls'
outp3 = r'C:\Users\achte\Documents\KI_Ba\DS_test\gps\gpx'

file_names = []

for path, subdir, files in os.walk(inp):
    for file in files:
        file_names.append(inp + "\\" + file)
        
# split file_names into 3 equal parts
len_names = len(file_names) // 3

files1 = file_names[:len_names]
files2 = file_names[len_names: len_names*2]
files3 = file_names[len_names:]

# save files
for file in files1:
    os.rename(file, outp1 + '\\' + file.split("\\")[-1])
    
for file in files2:
    os.rename(file, outp2 +  '\\' + file.split("\\")[-1])
    
for file in files3:
    os.rename(file, outp3 + '\\' + file.split("\\")[-1])


In [ ]:
#  convert to gpx

for file in file_names:
        # create dataframe
        df = pd.read_csv(file)

        # get filepath for output
        if outputdir:
            outf = outputdir + "\\" + file.split("\\")[-1].split(".")[0] + ".gpx"
        else:
            outf = file.split(".")[0] + ".gpx"

        # check if needed columns there (latitude, longitude)
        lat = ""
        long = ""
        for name in list(df.columns.values):
            if re.search('latitude|lat|LATITUDE|Latitude', name): lat = name
            if re.search('longitude|lon|LONGITUDE|Longitude', name): long = name
        if lat and long:
            Converter.dataframe_to_gpx(df, lats_colname=lat, longs_colname=long, output_file=outf)
        else:
            print(file + " is not a gps file")
            

In [ ]:
# convert to xls
from spire.xls import *
from spire.xls.common import *

for file in file_names:
        df = pd.read_csv(file)

        if outputdir:
            outf = outputdir + '\\' + file.split("\\")[-1].split(".")[0] + ".xlsx" 
            with pd.ExcelWriter(outf) as writer:
                df.to_excel(writer)
                
        outxls = outputdir + '\\' + file.split("\\")[-1].split(".")[0] + ".xls"
        workbook = Workbook()
        workbook.LoadFromFile(outf)
        workbook.SaveToFile(outxls, ExcelVersion.Version97to2003)
        workbook.Dispose()
        os.remove(outf)
        